In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import signals
import csv

class CatalogSpider(scrapy.Spider):
    name = "catalogdata"
    start_urls = ["http://dmoztools.net/Arts/Animation/Anime/Titles/3/3%C3%973_Eyes/",
                  "http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/"]
    itemlist = []

    @classmethod
    def from_crawler(cls, crawler):
        spider = super().from_crawler(crawler)
        crawler.signals.connect(spider.spider_closed, signals.spider_closed)
        return spider

    
    #csv writer
    def spider_closed(self):
        with open("dmoz.csv","w", newline="") as f:
            writer = csv.DictWriter(f,['Category','WebsiteName','WebsiteURL'])
            writer.writeheader()
            for data in self.itemlist:
                writer.writerow(data)

                
    # parser
    def parse(self, response):
        for record in response.css('.site-item'):
            items = {}
            items["Category"] = response.xpath('//title/text()').get()
            items["WebsiteName"] = record.css('.site-title::text').extract_first(default='')
            items["WebsiteURL"] = record.css('.title-and-desc a::attr(href)').extract_first(default='')
            
            # TO-DO
            # items[WebpageTitle]
            
            
            self.itemlist.append(items)

cat_spider = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0',   
})
cat_spider.crawl(CatalogSpider)
cat_spider.start()

:0: UserWarning: You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.
2020-04-04 11:46:52 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-04 11:46:52 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.5.2 (default, Oct  8 2019, 13:06:37) - [GCC 5.4.0 20160609], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.4.0-170-generic-x86_64-with-Ubuntu-16.04-xenial
2020-04-04 11:46:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-04-04 11:46:52 [scrapy.crawler] INFO: Overridden